<a href="https://colab.research.google.com/github/xinyanz-erin/Applied-Finance-Project/blob/Erin/v1_deep_learning_option_1_Erin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!curl https://colab.chainer.org/install |sh -
import cupy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0  24307      0 --:--:-- --:--:-- --:--:-- 24307
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 58.9MB 51kB/s 
     |████████████████████████████████| 1.0MB 36.1MB/s 
+ set +ex
Installation succeeded!


### Deep Learning Barrier Option

We used Numba and CuPy in the previous notebook to run Monte Carlo simulation to determine the price of the Asian Barrier option. A Monte Carlo simulation needs millions of paths to get an accurate answer which is computationally intensive. [Ryan et al (2018)](https://arxiv.org/abs/1809.02233) showed that a deep learning model can be trained to value derivatives. The deep learning model is accurate and very fast, capable of producing valuations a million times faster than traditional models. In the this notebook, we will use a fully connected network to learn the pricing mode of the Asian Barrier option. Monte Carlo simulation is used as pricing ground truth for the training. We use the same Asian Barrier Option model as last notebook with parameters listed as following:

```
T - Maturity (yrs.)
S - Spot (usd)
K - Strike (usd)
sigma - Volatility (per.)
r - Risk Free Rate (per.)
mu - Stock Drift Rate (per.)
B - Barrier (usd)
```

### Batched Data generation

The dataset is an important part of the Deep learning training. We will modify the previous single Asian Barrier Option pricing code to handle a batch of Barrier Option pricing. 

Loading all the necessary libraries:-

In [3]:
# !pip install cupy-cuda101

In [4]:
# import cupy
# import numpy as np
# import math
# import time
# import torch
# cupy.cuda.set_allocator(None)
# from torch.utils.dlpack import from_dlpack

The CuPy version of batched barrier option pricing simulation is as follows:-

In [5]:
# cupy_batched_barrier_option = cupy.RawKernel(r'''
# extern "C" __global__ void batched_barrier_option(
#     float *d_s,
#     const float T,
#     const float * K,
#     const float * B,
#     const float * S0,
#     const float * sigma,
#     const float * mu,
#     const float * r,
#     const float * d_normals,
#     const long N_STEPS,
#     const long N_PATHS,
#     const long N_BATCH)
# {
#   unsigned idx =  threadIdx.x + blockIdx.x * blockDim.x;
#   unsigned stride = blockDim.x * gridDim.x;
#   unsigned tid = threadIdx.x;
#   const float tmp3 = sqrt(T/N_STEPS);


#   for (unsigned i = idx; i<N_PATHS * N_BATCH; i+=stride)
#   {
#     int batch_id = i / N_PATHS;
#     int path_id = i % N_PATHS;
#     float s_curr = S0[batch_id];
#     float tmp1 = mu[batch_id]*T/N_STEPS;
#     float tmp2 = exp(-r[batch_id]*T);
#     unsigned n=0;
#     double running_average = 0.0;
#     for(unsigned n = 0; n < N_STEPS; n++){
#        s_curr += tmp1 * s_curr + sigma[batch_id]*s_curr*tmp3*d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH];
#        running_average += (s_curr - running_average) / (n + 1.0);
#        if (running_average <= B[batch_id]){
#            break;
#        }
#     }

#     float payoff = (running_average>K[batch_id] ? running_average-K[batch_id] : 0.f); 
#     d_s[i] = tmp2 * payoff;
#   }
# }

# ''', 'batched_barrier_option')

Note, the parameters (K, B, S0, sigma, mu, r) are passed in as an array with length of batch size. The output array is a two dimensional array flatten to 1-D. The first dimension is for Batch and the second dimension is for Path. 

Testing it out by entering two sets of option parameters:-

In [6]:
# N_PATHS = 2048000
# N_STEPS = 365
# N_BATCH = 2
# T = 1.0

# K = cupy.array([110.0, 120.0], dtype=cupy.float32)
# B = cupy.array([100.0, 90.0], dtype=cupy.float32)
# S0 = cupy.array([120.0, 100.0], dtype=cupy.float32)
# sigma = cupy.array([0.35, 0.2], dtype=cupy.float32)
# mu = cupy.array([0.15, 0.1], dtype=cupy.float32)
# r =cupy.array([0.05, 0.05], dtype=cupy.float32)

Put everything into a simple function to launch this GPU kernel. The option prices for each batch is the average of the corresponding path terminal values. This can be computed easily by Cupy function `mean(axis=1)`

In [7]:
# def batch_run():
#     number_of_threads = 256
#     number_of_blocks = (N_PATHS * N_BATCH - 1) // number_of_threads + 1
#     randoms_gpu = cupy.random.normal(0, 1, N_BATCH*N_PATHS * N_STEPS, dtype=cupy.float32)
#     output = cupy.zeros(N_BATCH*N_PATHS, dtype=cupy.float32)
#     cupy.cuda.stream.get_current_stream().synchronize()
#     s = time.time()
#     cupy_batched_barrier_option((number_of_blocks,), (number_of_threads,),
#                        (output, np.float32(T), K, B, S0, sigma, mu, r,
#                         randoms_gpu, N_STEPS, N_PATHS, N_BATCH))
#     v = output.reshape(N_BATCH, N_PATHS).mean(axis=1)
#     cupy.cuda.stream.get_current_stream().synchronize()
#     e = time.time()
#     print('time', e-s, 'v',v)
# batch_run()

This produces the option prices $21.22$ and $0.848$ for these two sets of option parameters in $66ms$.

It works efficiently hence we will construct an `OptionDataSet` class to wrap the above code so we can use it in Pytorch. For every `next` element, it generates uniform distributed random option parameters in the specified range, launches the GPU kernel to compute the option prices, convert the CuPy array to Pytorch tensors with zero copy via the DLPack. Note how we implemented the iterable Dataset interface:-

In [8]:
# class OptionDataSet(torch.utils.data.IterableDataset):
    
#     def __init__(self, max_len=10, number_path = 1000, batch=2, threads=256,seed=15):
#         self.num = 0
#         self.max_length = max_len
#         self.N_PATHS = number_path
#         self.N_STEPS = 365
#         self.N_BATCH = batch
#         self.T = np.float32(1.0)
#         self.output = cupy.zeros(self.N_BATCH*self.N_PATHS, dtype=cupy.float32) 
#         self.number_of_blocks = (self.N_PATHS * self.N_BATCH - 1) // threads + 1
#         self.number_of_threads = threads
#         cupy.random.seed(seed)
        
#     def __len__(self):
#         return self.max_length
        
#     def __iter__(self):
#         self.num = 0
#         return self
    
#     def __next__(self):
#         if self.num > self.max_length:
#             raise StopIteration
#         X = cupy.random.rand(self.N_BATCH, 6, dtype=cupy.float32)
#         # scale the [0, 1) random numbers to the correct range for each of the option parameters
#         X = X * cupy.array([200.0, 0.99, 200.0, 0.4, 0.2, 0.2], dtype=cupy.float32)
#         # make sure the Barrier is smaller than the Strike price
#         X[:, 1] = X[:, 0] * X[:, 1]
#         randoms = cupy.random.normal(0, 1, self.N_BATCH * self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
#         cupy_batched_barrier_option((self.number_of_blocks,), (self.number_of_threads,), (self.output, self.T, cupy.ascontiguousarray(X[:, 0]), 
#                               cupy.ascontiguousarray(X[:, 1]), cupy.ascontiguousarray(X[:, 2]), cupy.ascontiguousarray(X[:, 3]), cupy.ascontiguousarray(X[:, 4]), cupy.ascontiguousarray(X[:, 5]), randoms, self.N_STEPS, self.N_PATHS, self.N_BATCH))
#         Y = self.output.reshape(self.N_BATCH, self.N_PATHS).mean(axis=1)
#         self.num += 1
#         return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))

Put everything related to Pytorch dataset into a file `cupy_dataset.py`:-

In [9]:
# #%%writefile cupy_dataset.py 
# import cupy
# import numpy as np
# import torch
# from torch.utils.dlpack import from_dlpack
# cupy.cuda.set_allocator(None)

# cupy_batched_barrier_option = cupy.RawKernel(r'''
# extern "C" __global__ void batched_barrier_option(
#     float *d_s,
#     const float T,
#     const float * K,
#     const float * B,
#     const float * S0,
#     const float * sigma,
#     const float * mu,
#     const float * r,
#     const float * d_normals,
#     const long N_STEPS,
#     const long N_PATHS,
#     const long N_BATCH)
# {
#   unsigned idx =  threadIdx.x + blockIdx.x * blockDim.x;
#   unsigned stride = blockDim.x * gridDim.x;
#   unsigned tid = threadIdx.x;
#   const float tmp3 = sqrt(T/N_STEPS);


#   for (unsigned i = idx; i<N_PATHS * N_BATCH; i+=stride)
#   {
#     int batch_id = i / N_PATHS;
#     int path_id = i % N_PATHS;
#     float s_curr = S0[batch_id];
#     float tmp1 = mu[batch_id]*T/N_STEPS;
#     float tmp2 = exp(-r[batch_id]*T);
#     unsigned n=0;
#     double running_average = 0.0;
#     for(unsigned n = 0; n < N_STEPS; n++){
#        s_curr += tmp1 * s_curr + sigma[batch_id]*s_curr*tmp3*d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH];
#        running_average += (s_curr - running_average) / (n + 1.0);
#        if (running_average <= B[batch_id]){
#            break;
#        }
#     }

#     float payoff = (running_average>K[batch_id] ? running_average-K[batch_id] : 0.f); 
#     d_s[i] = tmp2 * payoff;
#   }
# }

# ''', 'batched_barrier_option')

# class OptionDataSet(torch.utils.data.IterableDataset):
    
#     def __init__(self, max_len=10, number_path = 1000, batch=2, threads=256,seed=15):
#         self.num = 0
#         self.max_length = max_len
#         self.N_PATHS = number_path
#         self.N_STEPS = 365
#         self.N_BATCH = batch
#         self.T = np.float32(1.0)
#         self.output = cupy.zeros(self.N_BATCH*self.N_PATHS, dtype=cupy.float32) 
#         self.number_of_blocks = (self.N_PATHS * self.N_BATCH - 1) // threads + 1
#         self.number_of_threads = threads
#         cupy.random.seed(seed)
        
#     def __len__(self):
#         return self.max_length
        
#     def __iter__(self):
#         self.num = 0
#         return self
    
#     def __next__(self):
#         if self.num > self.max_length:
#             raise StopIteration
#         X = cupy.random.rand(self.N_BATCH, 6, dtype=cupy.float32)
#         # scale the [0, 1) random numbers to the correct range for each of the option parameters
#         X = X * cupy.array([200.0, 0.99, 200.0, 0.4, 0.2, 0.2], dtype=cupy.float32)
#         # make sure the Barrier is smaller than the Strike price
#         X[:, 1] = X[:, 0] * X[:, 1]
#         randoms = cupy.random.normal(0, 1, self.N_BATCH * self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
#         cupy_batched_barrier_option((self.number_of_blocks,), (self.number_of_threads,), (self.output, self.T, cupy.ascontiguousarray(X[:, 0]), 
#                               cupy.ascontiguousarray(X[:, 1]), cupy.ascontiguousarray(X[:, 2]), cupy.ascontiguousarray(X[:, 3]), cupy.ascontiguousarray(X[:, 4]), cupy.ascontiguousarray(X[:, 5]), randoms, self.N_STEPS, self.N_PATHS, self.N_BATCH))
#         Y = self.output.reshape(self.N_BATCH, self.N_PATHS).mean(axis=1)
#         self.num += 1
#         return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))

Here is a test code to sample 10 data points with batch size 16:-

In [10]:
# from cupy_dataset import OptionDataSet
# ds = OptionDataSet(10, number_path=100000, batch=16, seed=15)
# for i in ds:
#     print(i[1])

We can implement the same code by using Numba to accelerate the calculation in GPU:-

In [11]:
# import numba
# from numba import cuda

# @cuda.jit
# def batch_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS, N_BATCH):
#     # ii - overall thread index
#     ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
#     stride = cuda.gridDim.x * cuda.blockDim.x
#     tmp3 = math.sqrt(T/N_STEPS)
#     for i in range(ii, N_PATHS * N_BATCH, stride):
#         batch_id = i // N_PATHS
#         path_id = i % N_PATHS
#         tmp1 = mu[batch_id]*T/N_STEPS
#         tmp2 = math.exp(-r[batch_id]*T)
#         running_average = 0.0
#         s_curr = S0[batch_id]
#         for n in range(N_STEPS):

#             s_curr += tmp1 * s_curr + sigma[batch_id]*s_curr*tmp3*d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH]
#             running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average)
#             if i==0 and batch_id == 2:
#                 print(s_curr)
#             if running_average <= B[batch_id]:
#                 break
#         payoff = running_average - K[batch_id] if running_average > K[batch_id] else 0
#         d_s[i] = tmp2 * payoff

# class NumbaOptionDataSet(object):
    
#     def __init__(self, max_len=10, number_path = 1000, batch=2, threads=512, seed=15):
#         self.num = 0
#         self.max_length = max_len
#         self.N_PATHS = number_path
#         self.N_STEPS = 365
#         self.N_BATCH = batch
#         self.T = np.float32(1.0)
#         self.output = cupy.zeros(self.N_BATCH*self.N_PATHS, dtype=cupy.float32) 
#         self.number_of_blocks = (self.N_PATHS * self.N_BATCH - 1) // threads + 1
#         self.number_of_threads = threads
#         cupy.random.seed(seed)
        
#     def __len__(self):
#         return self.max_length

#     def __iter__(self):
#         self.num = 0
#         return self
    
#     def __next__(self):
#         if self.num > self.max_length:
#             raise StopIteration
#         X = cupy.random.rand(self.N_BATCH, 6, dtype=cupy.float32)
#         # scale the [0, 1) random numbers to the correct range for each of the option parameters
#         X = X * cupy.array([200.0, 0.99, 200.0, 0.4, 0.2, 0.2], dtype=cupy.float32)
#         # make sure the Barrier is smaller than the Strike price
#         X[:, 1] = X[:, 0] * X[:, 1]
#         randoms = cupy.random.normal(0, 1, self.N_BATCH * self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
#         batch_barrier_option[(self.number_of_blocks,), (self.number_of_threads,)](self.output, self.T, X[:, 0], 
#                               X[:, 1], X[:, 2], X[:, 3], X[:, 4], X[:, 5], randoms, self.N_STEPS, self.N_PATHS, self.N_BATCH)
#         o = self.output.reshape(self.N_BATCH, self.N_PATHS)
#         Y = o.mean(axis = 1) 
#         self.num += 1
#         return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))
# ds = NumbaOptionDataSet(10, number_path=100000, batch=1, seed=15)
# for i in ds:
#     print(i[1])

# TEST_ERIN

In [13]:
################################# TEST ########################################
#%%writefile cupy_dataset.py

import numba
from numba import cuda
import random
import cupy
import numpy as np
import math
import time
import torch
cupy.cuda.set_allocator(None)
from torch.utils.dlpack import from_dlpack

# batch - number of underlyings in one basket option

@cuda.jit
def batch_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS, N_BATCH):
    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp3 = math.sqrt(T/N_STEPS)
    for i in range(ii, N_PATHS * N_BATCH, stride):
        batch_id = i // N_PATHS
        path_id = i % N_PATHS
        tmp1 = mu[batch_id]*T/N_STEPS
        tmp2 = math.exp(-r[batch_id]*T)
        running_average = 0.0
        s_curr = S0[batch_id]
        for n in range(N_STEPS):
            s_curr += tmp1 * s_curr + sigma[batch_id]*s_curr*tmp3*d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH] # stock price
            running_average = running_average + 1.0/(n + 1.0) * (s_curr - running_average) # average of the path
            if i==0 and batch_id == 2:
                print(s_curr)
            if running_average <= B[batch_id]: # if reach barrier, drop out the path
                break
        payoff = running_average - K[batch_id] if running_average > K[batch_id] else 0
        d_s[i] = tmp2 * payoff

class NumbaOptionDataSet(object):
    
    def __init__(self, max_len=10, number_path = 1000, batch=2, threads=512, seed=15, stocks=3):
        self.num = 0
        self.max_length = max_len
        self.N_PATHS = number_path
        self.N_STEPS = 365
        self.N_BATCH = batch
        self.N_STOCKS = stocks
        self.T = np.float32(1.0)
        self.output = cupy.zeros(self.N_STOCKS*self.N_PATHS, dtype=cupy.float32) 
        self.number_of_blocks = (self.N_PATHS * self.N_STOCKS - 1) // threads + 1
        self.number_of_threads = threads
        cupy.random.seed(seed)
        
    def __len__(self):
        return self.max_length
        
    def __iter__(self):
        self.num = 0
        return self
    
    def __next__(self):
        if self.num > self.max_length:
            raise StopIteration
        
        Y = cupy.zeros(self.N_BATCH, dtype=cupy.float32)
        paras = cupy.zeros((self.N_BATCH, self.N_STOCKS * 6), dtype = cupy.float32)
        for op in range(self.N_BATCH):
          # X = cupy.random.rand(self.N_STOCKS, 6, dtype=cupy.float32)
          X = cupy.array([200.0, 0.99, 200.0, 0.4, 0.2, 0.2] * self.N_STOCKS, dtype = cupy.float32).reshape(self.N_STOCKS, 6)
          # scale the [0, 1) random numbers to the correct range for each of the option parameters
          # X = X * cupy.array([200.0, 0.99, 200.0, 0.4, 0.2, 0.2], dtype=cupy.float32) # parameters
          # make sure the Barrier is smaller than the Strike price
          # X[:, 1] = X[:, 0] * X[:, 1]
          for i in range(self.N_STOCKS):
            paras[op, i*6:(i+1)*6] = X[i,:]

          stocks_randoms_mean = cupy.zeros(self.N_STOCKS, dtype = cupy.float32)
          rho = cupy.random.normal(0, 1, self.N_STOCKS, dtype = cupy.float32)
          #stocks_randoms_cov = cupy.ones((self.N_STOCKS, self.N_STOCKS), dtype = cupy.float32)
          #cupy.fill_diagonal(stocks_randoms_cov, rho)
          stocks_randoms_cov = (-0.99 + cupy.random.rand(self.N_STOCKS*self.N_STOCKS, dtype=cupy.float32)*2*0.99).reshape(self.N_STOCKS,self.N_STOCKS)
          cupy.fill_diagonal(stocks_randoms_cov, 1)
          num_of_randoms_each_stock = self.N_PATHS * self.N_STEPS
          randoms_gpu = cupy.random.multivariate_normal(stocks_randoms_mean, stocks_randoms_cov,
                                                        num_of_randoms_each_stock, dtype=cupy.float32)
          b1_r = randoms_gpu[:,0]
          b2_r = randoms_gpu[:,1]
          randoms = cupy.zeros(self.N_STOCKS * self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
          interval = int((self.N_PATHS * self.N_STEPS * self.N_STOCKS) / self.N_PATHS)
          for i in range(interval):
            if i % 2 == 0:
                ind = int(i/2)
                randoms[i*self.N_PATHS:(i+1)*self.N_PATHS] = b1_r[ind:(ind+self.N_PATHS)]
            else:
                ind = int(i//2)
                randoms[i*self.N_PATHS:(i+1)*self.N_PATHS] = b2_r[ind:(ind+self.N_PATHS)]

          randoms = cupy.random.normal(0, 1, self.N_STOCKS * self.N_PATHS * self.N_STEPS, dtype=cupy.float32)
          batch_barrier_option[(self.number_of_blocks,), (self.number_of_threads,)](self.output, self.T, X[:, 0], 
                                X[:, 1], X[:, 2], X[:, 3], X[:, 4], X[:, 5], randoms, self.N_STEPS, self.N_PATHS, self.N_STOCKS)
          
          o = self.output.reshape(self.N_STOCKS, self.N_PATHS)
          Y[op] = o.mean(axis = 0).mean()

        self.num += 1
        return (from_dlpack(paras.toDlpack()), from_dlpack(Y.toDlpack()))

ds = NumbaOptionDataSet(2, number_path=100000, batch=3, seed=random.randint(0,100), stocks=5)
for i in ds:
    print(i)
################################# TEST ########################################

(tensor([[200.0000,   0.9900, 200.0000,   0.4000,   0.2000,   0.2000, 200.0000,
           0.9900, 200.0000,   0.4000,   0.2000,   0.2000, 200.0000,   0.9900,
         200.0000,   0.4000,   0.2000,   0.2000, 200.0000,   0.9900, 200.0000,
           0.4000,   0.2000,   0.2000, 200.0000,   0.9900, 200.0000,   0.4000,
           0.2000,   0.2000],
        [200.0000,   0.9900, 200.0000,   0.4000,   0.2000,   0.2000, 200.0000,
           0.9900, 200.0000,   0.4000,   0.2000,   0.2000, 200.0000,   0.9900,
         200.0000,   0.4000,   0.2000,   0.2000, 200.0000,   0.9900, 200.0000,
           0.4000,   0.2000,   0.2000, 200.0000,   0.9900, 200.0000,   0.4000,
           0.2000,   0.2000],
        [200.0000,   0.9900, 200.0000,   0.4000,   0.2000,   0.2000, 200.0000,
           0.9900, 200.0000,   0.4000,   0.2000,   0.2000, 200.0000,   0.9900,
         200.0000,   0.4000,   0.2000,   0.2000, 200.0000,   0.9900, 200.0000,
           0.4000,   0.2000,   0.2000, 200.0000,   0.9900, 200.0000,  

### PUI TEST

In [ ]:
# %%writefile cupy_dataset.py
# import numba
# from numba import cuda
# import cupy
# import numpy as np
# import math
# import time
# import torch
# cupy.cuda.set_allocator(None)
# from torch.utils.dlpack import from_dlpack

# @cuda.jit
# def single_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS, N_STOCKS, s_curr):

#     # ii - overall thread index
#     ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
#     stride = cuda.gridDim.x * cuda.blockDim.x
#     tmp2 = math.exp(-r*T)
#     tmp3 = math.sqrt(T/N_STEPS)    

#     for i in range(ii, N_PATHS, stride): # for each path          
#         running_average = 0.0

#         for j in range(N_STOCKS): # initialize S0
#             s_curr[j] = S0[j]

#         for n in range(N_STEPS): # for each step
#             s_curr_avg = 0.0

#             for j in range(N_STOCKS): # for each stock
#                 tmp1 = mu[j]*T/N_STEPS  
#                 s_curr[j] += tmp1 * s_curr[j] + sigma[j]*s_curr[j]*tmp3*d_normals[i,n,j]
#                 s_curr_avg = s_curr_avg + 1.0/(j + 1.0) * (s_curr[j] - s_curr_avg) # S average in this step

#             # add stock average to running average
#             running_average = running_average + 1.0/(n + 1.0) * (s_curr_avg - running_average)

#             # compare to barrier
#             if running_average <= B:
#                 break

#         payoff = running_average - K if running_average > K else 0
#         d_s[i] = tmp2 * payoff
    

# class NumbaOptionDataSet(object):
    
#     def __init__(self, max_len=10, number_path = 1000, number_stocks = 3, batch=1, threads=512, seed=15, T=1):
#         self.num = 0
#         self.max_length = max_len
#         self.N_PATHS = number_path
#         self.N_STEPS = 365
#         self.N_STOCKS = number_stocks
#         self.N_BATCH = batch
#         self.T = np.float32(T)
#         self.output = cupy.zeros(self.N_PATHS, dtype=cupy.float32) 
#         self.number_of_blocks = (self.N_PATHS * self.N_STOCKS - 1) // threads + 1
#         self.number_of_threads = threads
#         cupy.random.seed(seed)

#         ############ <new
#         self.Z_mean = cupy.zeros(self.N_STOCKS, dtype=cupy.float32)
#         self.Z_cov = (-0.2 + cupy.random.rand(self.N_STOCKS*self.N_STOCKS, dtype=cupy.float32)*0.4).reshape(self.N_STOCKS,self.N_STOCKS)
#         cupy.fill_diagonal(self.Z_cov, 1)
#         ############ new>

#     def __len__(self):
#         return self.max_length
        
#     def __iter__(self):
#         self.num = 0
#         return self
    
#     def __next__(self):
#         if self.num > self.max_length:
#             raise StopIteration

#         X = cupy.zeros((self.N_BATCH, 3 + self.N_STOCKS * 3), dtype=cupy.float32)
#         Y = cupy.zeros(self.N_BATCH, dtype=cupy.float32)

#         for i in range(self.N_BATCH): # for each batch
#           self.S0 = cupy.random.rand(self.N_STOCKS, dtype=cupy.float32) * 200
#           self.K = 110.0
#           self.B = 100.0
#           self.sigma = cupy.random.rand(self.N_STOCKS, dtype=cupy.float32) * 0.2
#           self.mu = cupy.random.rand(self.N_STOCKS, dtype=cupy.float32) * 0.2
#           self.r = 0.05
#           self.s_curr = cupy.zeros(self.N_STOCKS, dtype=cupy.float32) # used to store s_curr in kernel

#           ############ <new - add correlation between stocks
#           all_normals = cupy.random.multivariate_normal(self.Z_mean, self.Z_cov, (self.N_PATHS, self.N_STEPS), dtype=cupy.float32)
#           ############ new>
          
#           single_barrier_option[(self.number_of_blocks,), (self.number_of_threads,)](self.output, self.T, self.K, self.B, self.S0, 
#                                                                                     self.sigma, self.mu, self.r, all_normals, self.N_STEPS, self.N_PATHS, self.N_STOCKS, self.s_curr)
#           Y[i] = self.output.mean()

#           ############ <new - combine to get X matrix
#           X[i,:] = cupy.array([self.K, self.B] + self.S0.tolist() +
#                                 self.sigma.tolist() + self.mu.tolist() + [self.r], dtype=cupy.float32)
#           ############ new>
        
#         self.num += 1
#         return (from_dlpack(X.toDlpack()), from_dlpack(Y.toDlpack()))

# ds = NumbaOptionDataSet(max_len=10, number_path=100, batch=2, seed=15)
# for i in ds:
#   print(i)

### Model
To map the option parameters to price, we use 6 layers of fully connected neural network with hidden dimension 512 as inspired by [this paper](https://arxiv.org/abs/1809.02233). Writing this DL price model into a file `model.py`:-

In [ ]:
# %%writefile model.py
# import torch.nn as nn
# import torch.nn.functional as F
# import torch


# class Net(nn.Module):

#     def __init__(self, hidden=1024):
#         super(Net, self).__init__()
#         self.fc1 = nn.Linear(6, hidden)
#         self.fc2 = nn.Linear(hidden, hidden)
#         self.fc3 = nn.Linear(hidden, hidden)
#         self.fc4 = nn.Linear(hidden, hidden)
#         self.fc5 = nn.Linear(hidden, hidden)
#         self.fc6 = nn.Linear(hidden, 1)
#         self.register_buffer('norm',
#                              torch.tensor([200.0,
#                                            198.0,
#                                            200.0,
#                                            0.4,
#                                            0.2,
#                                            0.2,]))

#     def forward(self, x):
#         # normalize the parameter to range [0-1] 
#         x = x / self.norm
#         x = F.elu(self.fc1(x))
#         x = F.elu(self.fc2(x))
#         x = F.elu(self.fc3(x))
#         x = F.elu(self.fc4(x))
#         x = F.elu(self.fc5(x))
#         return self.fc6(x)

### Modified Model

In [ ]:
%%writefile model.py
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np


class Net(nn.Module):

    def __init__(self, hidden=1024):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(18, hidden) # remember to change this!
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc4 = nn.Linear(hidden, hidden)
        self.fc5 = nn.Linear(hidden, hidden)
        self.fc6 = nn.Linear(hidden, 1)
        self.register_buffer('norm',
                             torch.tensor([200.0, 198.0, 200.0, 0.4, 0.2, 0.2,
                                           200.0, 198.0, 200.0, 0.4, 0.2, 0.2,
                                           200.0, 198.0, 200.0, 0.4, 0.2, 0.2])) # don't use numpy here - will give error later

    def forward(self, x):
        # normalize the parameter to range [0-1] 
        x = x / self.norm
        x = F.elu(self.fc1(x))
        x = F.elu(self.fc2(x))
        x = F.elu(self.fc3(x))
        x = F.elu(self.fc4(x))
        x = F.elu(self.fc5(x))
        return self.fc6(x)

Writing model.py


As we know the random parameters' scaling factors, the input parameters are first scaled back to a range of (0-1) by dividing them by (200.0, 198.0, 200.0, 0.4, 0.2, 0.2). Then they are projected 5 times to the hidden dimension of 512 after the `ELu` activation function. `ELu` is chosen because we need to compute the second order differentiation of the parameters. If use ReLu, the second order differentiation will always be zero. The last layer is a linear layer that maps the hidden dimension to the predicted option price. 

For training, we use [Ignite](https://github.com/pytorch/ignite) which is a high-level library to train neural networks in PyTorch. We use `MSELoss` as the loss function, `Adam` as the optimizer and `CosineAnnealingScheduler` as the learning rate scheduler. The following code is feeding the random option data to the pricing model to train it.

### Train

In [ ]:
!pip install pytorch-ignite

In [ ]:
from ignite.engine import Engine, Events
from ignite.handlers import Timer
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from model import Net
from cupy_dataset import NumbaOptionDataSet
timer = Timer(average=True)
model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
# dataset = NumbaOptionDataSet(max_len=10000, number_path = 1024, batch=4800)
dataset = NumbaOptionDataSet(max_len=100, number_path = 1024, batch=2, stocks=3)

def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    y_pred = model(x)
    loss = loss_fn(y_pred[:,0], y)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 100

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output, 'average time', timer.value())
        
trainer.run(dataset, max_epochs=100)

loss 0.0033376929350197315 average time 0.004678051810001307
loss 0.001502864295616746 average time 0.004562975160024507
loss 0.19722923636436462 average time 0.004543933840013778
loss 0.5098931789398193 average time 0.004573377949986935
loss 0.24777474999427795 average time 0.004604062109983716
loss 0.2487730234861374 average time 0.0045286981000253945
loss 0.6993882656097412 average time 0.00453493981997326
loss 0.1167130395770073 average time 0.00459125926001434
loss 0.057976506650447845 average time 0.004585574760012605
loss 0.2676795721054077 average time 0.004564111929980754
loss 0.024944454431533813 average time 0.004525489510001535
loss 0.9495897889137268 average time 0.004581687389973012
loss 0.677587628364563 average time 0.004581583870017312
loss 0.11046499013900757 average time 0.0045385417900206445
loss 0.23430660367012024 average time 0.004642491090003205
loss 0.05764923617243767 average time 0.004566168960009236
loss 0.3661995530128479 average time 0.004545230730022922
l

State:
	iteration: 10000
	epoch: 100
	epoch_length: 100
	max_epochs: 100
	output: 0.09467744827270508
	batch: <class 'tuple'>
	metrics: <class 'dict'>
	dataloader: <class 'cupy_dataset.NumbaOptionDataSet'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

The loss is keeping decreasing which means the pricing model can predict the option prices better. It takes about $12ms$ to compute one mini-batch in average, In the following sections, we will try to expore the full potentials of the GPU to accelerate the training.

### TensorCore mixed precision training

The V100 GPUs have 640 tensor cores that can accelerate half precision matrix multiplication calculation which is the core computation done by the DL model. [Apex library](https://github.com/NVIDIA/apex) developed by NVIDIA makes mixed precision and distributed training in Pytorch easy. By changing 3 lines of code, it can use the tensor cores to accelerate the training. 

In [ ]:
!git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 8054, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 8054 (delta 68), reused 97 (delta 44), pack-reused 7913
Receiving objects: 100% (8054/8054), 14.11 MiB | 25.99 MiB/s, done.
Resolving deltas: 100% (5467/5467), done.


In [ ]:
cd apex

/content/apex


In [ ]:
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-gbcnko0u
Created temporary directory: /tmp/pip-req-tracker-d_u2hq36
Created requirements tracker '/tmp/pip-req-tracker-d_u2hq36'
Created temporary directory: /tmp/pip-install-rfv4_uta
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-5mizzy79
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-d_u2hq36'
    Running setup.py (path:/tmp/pip-req-build-5mizzy79/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.9.0+cu102


    running egg_info
    creating /tmp/pip-req-build-5mizzy79/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-5mizzy79/pip-egg-info/apex.egg-info/PK

In [ ]:
from apex import amp
from ignite.engine import Engine, Events
from torch.nn import MSELoss
from ignite.handlers import Timer
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from model import Net
from cupy_dataset import NumbaOptionDataSet
timer = Timer(average=True)
model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
# set the AMP optimization level to O1
opt_level = 'O1'
# wrap the optimizer and model
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
#dataset = NumbaOptionDataSet(max_len=10000, number_path = 1024, batch=4800)
dataset = NumbaOptionDataSet(max_len=100, number_path = 1024, batch=2, stocks=3)

def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    y_pred = model(x)
    loss = loss_fn(y_pred[:,0], y)
    # amp handles the auto loss scaling
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 100
timer.attach(trainer,
             start=Events.EPOCH_STARTED,
             resume=Events.ITERATION_STARTED,
             pause=Events.ITERATION_COMPLETED,
             step=Events.ITERATION_COMPLETED)    
scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output, 'average time', timer.value())
        
trainer.run(dataset, max_epochs=100)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 40

State:
	iteration: 10000
	epoch: 100
	epoch_length: 100
	max_epochs: 100
	output: 0.0883660614490509
	batch: <class 'tuple'>
	metrics: <class 'dict'>
	dataloader: <class 'cupy_dataset.NumbaOptionDataSet'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

It improves to compute each mini-batch in $8ms$. As we reduce the model weights to half precision for better performance, the loss need to be scaled to make sure the half precision dynamic range aligns with the computation. It is guessing what is the correct loss scaling factor and adjust it automatically if the gradient overflows. In the end, we will get the best hardware acceleration while maintaining the accuracy of model prediction.

### Multiple GPU training

Apex makes multiple GPU training easy. Working on the same training script, we need to take care of a few extra steps:

1. Add the argument `--local_rank` which will be automatically set by the distributed launcher
2. Initialize the process group
2. Generate independent batched data based on process id in the dataset.
3. Wrap the model and optimizer to handle distributed computation. 
4. Scale the loss and optimizer

To launch distributed training, we need to put everything into a python file. Following is an example:-

In [ ]:
cd ..

/content


In [ ]:
%%writefile distributed_train.py 
import cupy
import numpy as np
import math
import time
import os
import torch
from torch.utils.dlpack import from_dlpack
import torch.nn as nn
import torch.nn.functional as F
import torch
from apex import amp
from ignite.engine import Engine, Events
from torch.nn import MSELoss
from torch.optim import Adam
from ignite.contrib.handlers.param_scheduler import CosineAnnealingScheduler
from ignite.handlers import ModelCheckpoint
from apex.parallel import DistributedDataParallel 
import argparse
from model import Net
from cupy_dataset import NumbaOptionDataSet

parser = argparse.ArgumentParser()
parser = argparse.ArgumentParser()
# this local_rank arg is automaticall set by distributed launch
parser.add_argument("--local_rank", default=0, type=int)
args = parser.parse_args()

args.distributed = False
if 'WORLD_SIZE' in os.environ:
    args.distributed = int(os.environ['WORLD_SIZE']) > 1

if args.distributed:
    torch.cuda.set_device(args.local_rank)
    torch.distributed.init_process_group(backend='nccl',
                                         init_method='env://')

torch.backends.cudnn.benchmark = True


model = Net().cuda()
loss_fn = MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)
opt_level = 'O1'
model, optimizer = amp.initialize(model, optimizer, opt_level=opt_level)
if args.distributed:
    model = DistributedDataParallel(model)
#dataset = NumbaOptionDataSet(max_len=10000, number_path = 1024, batch=10240, seed=args.local_rank)
dataset = NumbaOptionDataSet(max_len=100, number_path = 1024, batch=2, stocks=3, seed=args.local_rank)

def train_update(engine, batch):
    model.train()
    optimizer.zero_grad()
    x = batch[0]
    y = batch[1]
    y_pred = model(x)
    loss = loss_fn(y_pred[:,0], y)
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(train_update)
log_interval = 100

scheduler = CosineAnnealingScheduler(optimizer, 'lr', 1e-4, 1e-6, len(dataset))
trainer.add_event_handler(Events.ITERATION_STARTED, scheduler)
    
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iter = (engine.state.iteration - 1) % len(dataset) + 1
    if iter % log_interval == 0:
        print('loss', engine.state.output)
        
trainer.run(dataset, max_epochs=100)

Writing distributed_train.py


To launch multiple processes training, we need to run the following command:-

In [ ]:
%reset -f

!python -m torch.distributed.launch --nproc_per_node=4 distributed_train.py

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:164: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  "The module torch.distributed.launch is deprecated "
The module torch.distributed.launch is deprecated and going to be removed in future.Migrate to torch.distributed.run
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
 Please read local_rank from `os.environ('LOCAL_RANK')` instead.
INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
  entrypoint       : distributed_train.py
  min_nodes        : 1
  max_nodes        : 1
  nproc_per_node   : 4
  run_id           : none
  rdzv_backend     : static
  rdzv_endpoint    : 127.0.0.1:29500
  rdzv_configs     : {'rank': 0, 'tim

It works and all the GPUs are busy to train this network. However, it has a few problems:-
   
    1. There is no model serialization so the trained model is not saved
    2. There is no validation dataset to check the training progress
    3. Most of the time is spent in Monte Carlo simulation hence the training is slow
    4. We use a few paths(1024) for each option parameter set which is noise and the model cannot converge to a low cost value.
We will address these problems in the next notebook